In [1]:
# import dependencies 
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
import os
import random

In [27]:

# open browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 104.0.5112
Get LATEST chromedriver version for 104.0.5112 google-chrome
Driver [/Users/evanbruno/.wdm/drivers/chromedriver/mac64/104.0.5112.79/chromedriver] found in cache


In [28]:
# go to source website
url = "https://www.rentdata.org/states/california/2006"
browser.visit(url)

In [29]:
html = browser.html
test = soup(html, 'html.parser')

In [30]:
# get headings for columns
headings = []
for head in test.find("thead").find_all("th"):
    headings.append(head.get_text())

In [53]:
# get county names into dict
counties = {}
for data in test.find_all("tbody")[1].find_all("tr"):
    county = data.find("td").get_text()
    counties[county] = {}

In [54]:
# get first year 2006
for county in counties:
    counties[county]["2006"] = []

In [55]:
# get data for 2006
for data in test.find_all("tbody")[1].find_all("tr"):
    county = data.find("td").get_text()
    for i in data.find_all("td")[1:]:
        counties[county]["2006"].append(i.get_text())

In [56]:
counties

{'Alameda County\nMetro\n': {'2006': ['$865',
   '$1,045',
   '$1,238',
   '$1,679',
   '$2,079',
   '1,443,741']},
 'Alpine County\n': {'2006': ['$498',
   '$559',
   '$713',
   '$1,017',
   '$1,047',
   '1,208']},
 'Amador County\n': {'2006': ['$537',
   '$629',
   '$826',
   '$1,200',
   '$1,237',
   '35,100']},
 'Butte County\nMetro\n': {'2006': ['$473',
   '$562',
   '$678',
   '$956',
   '$1,141',
   '203,171']},
 'Calaveras County\n': {'2006': ['$548',
   '$548',
   '$659',
   '$961',
   '$1,060',
   '40,554']},
 'Colusa County\n': {'2006': ['$500',
   '$502',
   '$653',
   '$843',
   '$1,146',
   '18,804']},
 'Contra Costa County\nMetro\n': {'2006': ['$865',
   '$1,045',
   '$1,238',
   '$1,679',
   '$2,079',
   '948,816']},
 'Del Norte County\n': {'2006': ['$490',
   '$496',
   '$642',
   '$935',
   '$964',
   '27,507']},
 'El Dorado County\nMetro\n': {'2006': ['$691',
   '$786',
   '$959',
   '$1,384',
   '$1,586',
   '156,299']},
 'Fresno County\nMetro\n': {'2006': ['$540',


In [57]:
# obtain href for other years
hrefs = []
for i in test.find_all(class_="button-group")[0].find_all("a")[1:-1]:
    hrefs.append(i.get("href"))

In [59]:
#iterate through hrefs
for year in hrefs:
    time.sleep(2)
    browser.visit(year)
    html = browser.html
    test = soup(html, 'html.parser')
    
    #get data for each county
    for county in counties:
        counties[county][year[-4:]] = []
    for data in test.find_all("tbody")[1].find_all("tr"):
        county = data.find("td").get_text()
        for i in data.find_all("td")[1:]:
            counties[county][year[-4:]].append(i.get_text())

In [60]:
counties

{'Alameda County\nMetro\n': {'2006': ['$865',
   '$1,045',
   '$1,238',
   '$1,679',
   '$2,079',
   '1,443,741'],
  '2007': ['$874', '$1,055', '$1,250', '$1,695', '$2,099', '1,443,741'],
  '2008': ['$866', '$1,046', '$1,239', '$1,680', '$2,080', '1,443,741'],
  '2009': ['$905', '$1,093', '$1,295', '$1,756', '$2,174', '1,443,741'],
  '2010': ['$963', '$1,162', '$1,377', '$1,867', '$2,312', '1,443,741'],
  '2011': ['$974', '$1,176', '$1,393', '$1,889', '$2,339', '1,443,741'],
  '2012': ['$980', '$1,183', '$1,402', '$1,901', '$2,354', '1,443,741'],
  '2013': ['$892', '$1,082', '$1,361', '$1,901', '$2,332', '1,510,271'],
  '2014': ['$1,035', '$1,255', '$1,578', '$2,204', '$2,704', '1,510,271'],
  '2015': ['$1,039', '$1,260', '$1,585', '$2,213', '$2,716', '1,510,271'],
  '2016': ['$1,380', '$1,663', '$2,103', '$2,932', '$3,268', '1,510,271'],
  '2017': ['$1,435', '$1,723', '$2,173', '$3,017', '$3,477', '1,510,271'],
  '2018': ['$1,540', '$1,855', '$2,329', '$3,219', '$3,946', '1,510,271'],

In [70]:
# convert county to csv
for county in counties:
    data = pd.DataFrame.from_dict(counties["Alameda County\nMetro\n"], orient = "index", columns = headings[1:])
    name = county.split("\n")[0]
    
    data.to_csv(f"Counties_Rent/{name}.csv")

In [20]:
browser.quit()